# How well do other distributions approximate the von Mises?

In [ ]:
import math
from random import random

import numpy as np
import torch
from torch.distributions import (
    constraints,
    Transform,
    TransformedDistribution,
    Normal,
    Uniform,
    VonMises,
)
import matplotlib.pyplot as plt

import utils

%load_ext lab_black

## Uniform

In [ ]:
concentrations = torch.linspace(0.01, 0.1, 24)
acceptances = []
for conc in concentrations:
    acceptances.append(
        utils.metropolis_acceptance(
            Uniform(low=-math.pi, high=math.pi),
            VonMises(loc=0, concentration=conc),
        )
    )
fig, ax = plt.subplots()
ax.plot(concentrations, acceptances, "o-")
ax.set_xlabel("concentration, rho")
ax.set_ylabel("Metropolis acceptance")
ax.set_title("Uniform distribution")
fig.tight_layout()
#fig.savefig("acceptance_unif_vonmises_0.01-0.1.png")

## Gaussian

In [ ]:
concentrations = torch.linspace(10, 34, 24)
acceptances = []
for conc in concentrations:
    acceptances.append(
        utils.metropolis_acceptance(
            Normal(loc=0, scale=math.sqrt(1 / conc)),
            VonMises(loc=0, concentration=conc),
        )
    )
fig, ax = plt.subplots()
ax.plot(concentrations, acceptances, "o-")
ax.set_xlabel("concentration, rho")
ax.set_ylabel("Metropolis acceptance")
ax.set_title("Gaussian")
fig.savefig("acceptance_gauss_vonmises.png")

## Wrapped Cauchy

In [ ]:
class MobiusTransform(Transform):
    bijective = True
    domain = constraints.interval(-math.pi, math.pi)
    codomain = constraints.interval(-math.pi, math.pi)

    def __init__(self, rho):
        super().__init__()
        self.rho = rho

    def _call(self, z):
        return 2 * torch.atan((1 - self.rho) / (1 + self.rho) * torch.tan(z / 2))

    def _inverse(self, y):
        return 2 * torch.atan((1 + self.rho) / (1 - self.rho) * torch.tan(y / 2))

    def log_abs_det_jacobian(self, z, y):
        return -torch.log(
            (1 - self.rho.pow(2)) / (1 + self.rho.pow(2) - 2 * self.rho * torch.cos(y))
        )


concentrations = torch.linspace(1, 10, 24)
acceptances = []
for conc in concentrations:
    acceptances.append(
        metropolis_acceptance(
            TransformedDistribution(
                base_distribution=Uniform(low=-math.pi, high=math.pi),
                transforms=MobiusTransform(torch.exp(-1 / conc.sqrt())),
            ),
            VonMises(loc=0, concentration=conc),
        )
    )
fig, ax = plt.subplots()
ax.plot(concentrations, acceptances, "o-")
ax.set_xlabel("concentration, rho")
ax.set_ylabel("Metropolis acceptance")
ax.set_title("Wrapped Cauchy")
fig.savefig("acceptance_cauchy_vonmises_1-10.png")